In [2]:
from datetime import datetime, timedelta
import os
import pandas as pd
import requests
from sqlalchemy import create_engine
%run config.ipynb

In [3]:
stock = pd.read_csv('assets/data/WMT.csv')
stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1972-08-25,0.063477,0.064697,0.063477,0.064453,0.023123,2508800
1,1972-08-28,0.064453,0.064941,0.064209,0.064209,0.023035,972800
2,1972-08-29,0.063965,0.063965,0.063477,0.063477,0.022772,1945600
3,1972-08-30,0.063477,0.063477,0.062988,0.063477,0.022772,409600
4,1972-08-31,0.062988,0.062988,0.062500,0.062500,0.022422,870400


In [4]:
Color=[]
Difference=[]
for i in range(len(stock)):
    Difference.append(stock["Close"][i]-stock["Open"][i])
    if Difference[i]>=0:
        Color.append("#8b0000")
    
    elif Difference[i]<0:
        Color.append("#006400")
stock["Difference"]=Difference
stock["Color"]=Color
stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Difference,Color
0,1972-08-25,0.063477,0.064697,0.063477,0.064453,0.023123,2508800,0.000976,#8b0000
1,1972-08-28,0.064453,0.064941,0.064209,0.064209,0.023035,972800,-0.000244,#006400
2,1972-08-29,0.063965,0.063965,0.063477,0.063477,0.022772,1945600,-0.000488,#006400
3,1972-08-30,0.063477,0.063477,0.062988,0.063477,0.022772,409600,0.000000,#8b0000
4,1972-08-31,0.062988,0.062988,0.062500,0.062500,0.022422,870400,-0.000488,#006400


In [5]:
stock_df = stock.loc[stock['Date'] >="2010-01-01", :]
stock_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Difference,Color
9425,2010-01-04,53.740002,54.669998,53.669998,54.230000,41.552284,20753100,0.489998,#8b0000
9426,2010-01-05,54.090000,54.189999,53.570000,53.689999,41.138515,15648400,-0.400001,#006400
9427,2010-01-06,53.500000,53.830002,53.419998,53.570000,41.046570,12517200,0.070000,#8b0000
9428,2010-01-07,53.720001,53.750000,53.259998,53.599998,41.069557,10662700,-0.120003,#006400
9429,2010-01-08,53.430000,53.529999,53.020000,53.330002,40.862682,11363200,-0.099998,#006400


In [6]:
moving_avg= stock_df['Difference'].rolling(window=180).mean()
stock_df["MovingAvg"]=moving_avg
stock_df.head()


<ipython-input-6-7f1b8a86812a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_df["MovingAvg"]=moving_avg


,Date,Open,High,Low,Close,Adj Close,Volume,Difference,Color,MovingAvg
9425,2010-01-04,53.740002,54.669998,53.669998,54.230000,41.552284,20753100,0.489998,#8b0000,NaN
9426,2010-01-05,54.090000,54.189999,53.570000,53.689999,41.138515,15648400,-0.400001,#006400,NaN
9427,2010-01-06,53.500000,53.830002,53.419998,53.570000,41.046570,12517200,0.070000,#8b0000,NaN
9428,2010-01-07,53.720001,53.750000,53.259998,53.599998,41.069557,10662700,-0.120003,#006400,NaN
9429,2010-01-08,53.430000,53.529999,53.020000,53.330002,40.862682,11363200,-0.099998,#006400,NaN


In [7]:
outputfile = "WalmartStock.csv"
stock_df.to_csv(outputfile, header = True, index=False)

In [9]:
import psycopg2

engine=create_engine(f"postgres:{connection_string}")

In [10]:
stock_df.to_sql(name="stock", con=engine, if_exists='append', index=False)

In [11]:
engine.execute('ALTER TABLE "stock" ADD PRIMARY KEY ("Date")')